In [2]:
import boto3
import pandas as pd
import json
from io import StringIO

In [3]:
s3_bucket = "ads-508-final"
s3_key = "playstation/games/games.csv"

In [4]:
s3 = boto3.client("s3")

# Read the CSV file from S3
response = s3.get_object(Bucket=s3_bucket, Key=s3_key)
csv_data = response["Body"].read().decode("utf-8")

# Load CSV into a Pandas DataFrame
df = pd.read_csv(StringIO(csv_data))

In [5]:
def fix_array(value):
    """Converts a Python-style string list to a proper JSON array"""
    if pd.isna(value) or value == "":
        return "[]"  # Empty array
    return json.dumps(eval(value))  # Convert to JSON format

# Apply fix to all array columns
array_columns = ['developers', 'publishers', 'genres', 'supported_languages']
for col in array_columns:
    df[col] = df[col].apply(fix_array)

In [6]:
# Convert DataFrame to CSV in memory
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)

# Upload back to S3 (overwrite the existing file)
s3.put_object(Bucket=s3_bucket, Key=s3_key, Body=csv_buffer.getvalue())

print(f"File successfully updated in s3://{s3_bucket}/{s3_key}")

File successfully updated in s3://ads-508-final/playstation/games/games.csv
